In [1]:
import argparse
import torch
import dataloader

In [2]:
from torchvision.transforms import ToTensor

In [3]:
from torch.utils.data import random_split

In [4]:
from torch.utils.data import DataLoader

In [5]:
import unet

In [6]:
from unet.unet_model import UNet

In [9]:
from torchvision import models
from torchsummary import summary

In [10]:
torch.cuda.is_available()

True

In [11]:
ap = argparse.ArgumentParser()
ap.add_argument("-m", "--model", type = str, required = True,
                help = "path to output trained model")
ap.add_argument("-p", "--plot", type = str, required = True,
                help = "path to output loss/accuracy plot")
args = vars(ap.parse_args())


usage: ipykernel_launcher.py [-h] -m MODEL -p PLOT
ipykernel_launcher.py: error: the following arguments are required: -m/--model, -p/--plot


SystemExit: 2

In [13]:
# define training parameters
INIT_LR = 1e-3
BATCH_SIZE = 1
EPOCHS = 10

# define the train and val splits
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

# set the device we will be using to train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# load the dataset
print("[INFO] loading the paired desmoke image dataset...")
dataset = dataloader.PairedSmokeImageDataset(
    csv_file = '..\\..\\datasets\\DesmokeData-paired\\DesmokeData-main\\images\\paired_images.csv',
    root_dir = '..\\..\\datasets\\DesmokeData-paired\\DesmokeData-main\\images\\dataset',
    transform = ToTensor())

num_train_samples = int(len(dataset) * TRAIN_SPLIT) + 1
num_val_samples = int(len(dataset) * VAL_SPLIT)
num_test_samples = int(len(dataset) * TEST_SPLIT)

# print(num_train_samples)
# print(num_val_samples)
# print(num_test_samples)

(train_data, val_data, test_data) = random_split(dataset, [num_train_samples, num_val_samples, num_test_samples],
                                                 generator=torch.Generator().manual_seed(42))
# print(len(train_data))
# print(len(val_data))
# print(len(test_data))




[INFO] loading the paired desmoke image dataset...


In [21]:
# initialize the train, validation and test data loaders
train_dataloader = DataLoader(train_data, shuffle = True, batch_size = BATCH_SIZE)
val_dataloader = DataLoader(val_data, batch_size = BATCH_SIZE)
test_dataloader = DataLoader(test_data, batch_size = BATCH_SIZE)

# calculate steps per epoch for training and validation set
train_steps = len(train_dataloader.dataset) // BATCH_SIZE
val_steps = len(val_dataloader.dataset) // BATCH_SIZE

In [22]:
# initialize the u-net model
print("[INFO] initializing the U-Net model ...")
# model = UNet()

[INFO] initializing the U-Net model ...


In [23]:
unet = UNet(n_channels=3, n_classes=2).cuda()
summary(unet, (3,700,350))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 700, 350]           1,728
       BatchNorm2d-2         [-1, 64, 700, 350]             128
              ReLU-3         [-1, 64, 700, 350]               0
            Conv2d-4         [-1, 64, 700, 350]          36,864
       BatchNorm2d-5         [-1, 64, 700, 350]             128
              ReLU-6         [-1, 64, 700, 350]               0
        DoubleConv-7         [-1, 64, 700, 350]               0
         MaxPool2d-8         [-1, 64, 350, 175]               0
            Conv2d-9        [-1, 128, 350, 175]          73,728
      BatchNorm2d-10        [-1, 128, 350, 175]             256
             ReLU-11        [-1, 128, 350, 175]               0
           Conv2d-12        [-1, 128, 350, 175]         147,456
      BatchNorm2d-13        [-1, 128, 350, 175]             256
             ReLU-14        [-1, 128, 3